# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.114016e+02     1.625323e+00
 * time: 0.12853789329528809
     1     1.059104e+01     9.787446e-01
 * time: 1.0091850757598877
     2    -1.057054e+01     1.017210e+00
 * time: 1.154404878616333
     3    -3.332094e+01     8.315349e-01
 * time: 1.315176010131836
     4    -4.650909e+01     7.051791e-01
 * time: 1.4667038917541504
     5    -5.644754e+01     2.723994e-01
 * time: 1.6083869934082031
     6    -5.945460e+01     2.452829e-01
 * time: 1.726240873336792
     7    -6.071587e+01     1.093965e-01
 * time: 1.846817970275879
     8    -6.125508e+01     4.079570e-02
 * time: 1.9587469100952148
     9    -6.157361e+01     2.983654e-02
 * time: 2.0774478912353516
    10    -6.176891e+01     3.854248e-02
 * time: 2.1883790493011475
    11    -6.189098e+01     2.813712e-02
 * time: 2.3076589107513428
    12    -6.198773e+01     2.189893e-02
 * time: 2.427854061126709
    13    -6.206206e+01     1.499662e-02
 * time: 2.53834390640258

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671067
    AtomicLocal         -18.8557676
    AtomicNonlocal      14.8522645
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485371 
    Xc                  -19.3336820

    total               -62.261666462534
